# kubectl 명령어 모음

### kubectl get all
**특정 Kubernetes 네임스페이스의 리소스를 확인하는 명령어**

In [ ]:
!kubectl get all

- **ml-pipeline-ui-artifact**:
    - **역할**: 머신러닝 파이프라인의 아티팩트를 시각화하는 데 사용되는 웹 UI 컴포넌트
    - **기능:** 파이프라인 실행 결과로 생성된 아티팩트(예: 모델, 데이터셋, 로그 등)를 웹 인터페이스를 통해 확인하고, 분석할 수 있는 기능을 제공함.
- **ml-pipeline-visualizationserver**:
    - **역할**: 파이프라인 실행 결과를 시각화하는 서버
    - **기능**: 파이프라인의 결과를 다양한 형태의 시각화(예: 차트, 그래프 등)로 제공하기 위해 필요한 데이터를 처리하고, 이를 UI 컴포넌트로 전달함.
- **Notebook 서버(StatefulSets)**:
    - Kubeflow의 Notebook 서버는 일반적으로 StatefulSet을 사용하여 배포됨.
    - Statefulset으로 생성되는 이유
        - **Persistent Storage**: Notebooks는 일반적으로 데이터를 저장할 수 있어야 하기 때문에 PersistentVolumeClaims(PVCs)를 사용함.
        - Notebook Pod는 고유한 네트워크 ID를 가지므로, 다시 시작되거나 스케일링될 때도 동일한 네트워크 ID를 유지함. 이는 사용자가 Notebooks에 접근할 때 중요한 요소임.
        - StatefulSet은 Pod의 생성, 삭제 및 업데이트를 순서대로 처리함. Notebook 서버와 같은 상태를 유지해야 하는 애플리케이션에 적합함.

### kubectl get sts -o yaml
**Kubernetes StatefulSet 리소스를 YAML 형식으로 출력하는 명령어**

In [ ]:
!kubectl get sts -o yaml

- **kind: StatefulSet**
    - 리소스의 종류를 나타냄. 현재 StatefulSet으로, 이는 상태가 있는 애플리케이션을 관리하는 Kubernetes 컨트롤러임.
- **metadata.namespace: kbm-u-kubeflow-tutorial**
    - 리소스가 속한 네임스페이스를 나타냄. 여기서는 kbm-u-kubeflow-tutorial 네임스페이스에 속함.
- **spec.replicas: 1**
    - 생성할 Pod의 복제본 수를 지정함. 여기서는 1개의 복제본이 생성되도록 설정함.
- **spec.template.spec.affinity**
    - Pod가 특정 노드 풀에서 실행되도록 노드 어피니티를 설정함. 여기서는 kakaoi.io/kke-nodepool 키와 pool-worker 값을 가진 노드에 스케줄링되도록 설정됨.
- **containers.image: bigdata.kr-central-1.kcr.dev/mlops-pipelines/jupyter-tensorflow-full:v1.0.1.py38**
    - 컨테이너에서 사용할 Docker 이미지를 지정함. 여기서는 Jupyter Notebook을 실행하기 위한 이미지 bigdata.kr-central-1.kcr.dev/mlops-pipelines/jupyter-tensorflow-full:v1.0.1.py38가 사용됨.
- **volumeMounts:**
    - 여러 볼륨을 특정 경로에 마운트함. 여기서는 /var/run/docker.sock, /dev/shm, /home/jovyan/vol-2, /home/jovyan 경로에 볼륨이 마운트됨.
- **serviceAccountName: default-editor**
    - 이 StatefulSet에서 사용하는 ServiceAccount의 이름을 지정함. 여기서는 default-editor가 사용됨.
- **spec.template.spec.volumes:**
    - StatefulSet에서 사용하는 볼륨을 정의함. 여기서는 persistentVolumeClaim으로 정의된 test와 test-volume 볼륨이 사용됨.

### kubectl get sa
**Kubernetes ServiceAccount 리소스를 조회하는 명령어**

In [ ]:
!kubectl get sa

**`ServiceAccount`** 는 Kubernetes 내의 파드(Pod)에서 실행되는 애플리케이션이 Kubernetes API 서버에 접근할 때 사용하는 계정임. 파드가 특정 작업을 수행할 수 있도록 제한된 권한을 부여함
    


- 매핑된 바인딩 확인하기(데모 - 포함할지 말지 아직 안정해짐)

In [ ]:
# kbm-u-kubeflow-tutorial Namespace에서 RoleBinding 리소스를 조회하는 명령어입니다.
!kubectl get rolebinding -n kbm-u-kubeflow-tutorial

In [ ]:
# kbm-u-kubeflow-tutorial Namespace에서 default-editor라는 이름의 RoleBinding 리소스를 YAML 형식으로 조회하는 명령어입니다.
!kubectl get rolebinding default-editor -n kbm-u-kubeflow-tutorial -o yaml

In [ ]:
# kbm-u-kubeflow-tutorial Namespace에서 default-viewer라는 이름의 RoleBinding 리소스를 YAML 형식으로 조회하는 명령어입니다.
!kubectl get rolebinding default-viewer -n kbm-u-kubeflow-tutorial -o yaml

- **`default-editor`** 서비스 계정은 Kubeflow에서 편집 권한을 가진 사용자를 위한 계정임. 클러스터 내에서 리소스를 생성, 수정, 삭제할 수 있는 권한을 가짐
    - 새로운 노트북 서버 생성
    - 실험, 실행, 파이프라인 관리
    - 데이터셋 및 모델의 생성, 수정, 삭제
    - 기타 클러스터 리소스에 대한 쓰기 작업

- **`default-viewer`** 서비스 계정은 읽기 전용 권한을 가진 사용자를 위한 계정임. 클러스터 내에서 리소스를 조회할 수 있지만, 생성, 수정, 삭제는 할 수 없음.
    - 노트북 서버 조회
    - 실험, 실행, 파이프라인 조회
    - 데이터셋 및 모델 조회
    - 기타 클러스터 리소스에 대한 읽기 작업

### kubectl get pvc -o yaml
**현재 Namespace에서 모든 PersistentVolumeClaim(PVC) 리소스를 YAML 형식으로 조회하는 명령어**

In [ ]:
!kubectl get pvc

```bash
    (base) jovyan@test-0:~$ kubectl get pvc
    NAME          STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS                         AGE
    test-volume   Bound    pvc-394a65be-d36a-4ee3-b7e1-a4ab45712e4a   10Gi       RWO            nfs-kbm-u-kubeflow-tutorial-client   31h
```
- **`test-volume`** VPC는 **`Bound`** 상태임.
- **`test-volume`** 의  용량은 10Gi임.
- **`RWO`**(ReadWriteOnce) 액세스 모드 → 특정 시점에 오직 하나의 컴퓨터(노드)만 이 볼륨에 접근함.
- **`nfs-kbm-u-kubeflow-tutorial-client`** 라는 스토리지 클래스를 사용 → PVC들이 NFS(Network File System)를 기반으로 한 스토리지 클래스를 사용함을 의미함.